## Preprocessing

In [3]:
# Import our dependencies
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from keras.layers import Dense, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
import numpy as numpy
import h5py

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

2024-06-11 20:48:52.219759: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

application_df.drop(["EIN","NAME"],axis =1, inplace =True)
application_df.head(2)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1


In [5]:
# Determine the number of unique values in each column.

application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"

application_df["APPLICATION_TYPE"].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

cutoff = 276
value_counts=application_df["APPLICATION_TYPE"].value_counts()
application_types_to_replace = value_counts[value_counts <= cutoff].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"

application_df["CLASSIFICATION"].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1

classification_counts = application_df["CLASSIFICATION"].value_counts()
classification_counts=classification_counts[classification_counts >1]
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classification_counts = application_df['CLASSIFICATION'].value_counts()
cutoff = 1883  # Example cutoff, adjust as needed based on the actual data

# Create a list of classifications to replace
classifications_to_replace = classification_counts[classification_counts < cutoff].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [11]:

# Convert categorical data to numeric with `pd.get_dummies`

application_df =pd.get_dummies(application_df)
application_df.head(2)
application_df = application_df.astype(int)
application_df.head(2)

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [12]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(["IS_SUCCESSFUL"],axis = 1)
y= application_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


## Compile, Train and Evaluate the Model

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(units =80, activation = "relu", input_dim =43))

# Second hidden layer
nn.add(Dense(units =30, activation ="relu"))

# Output layer
nn.add(Dense( units =1 , activation ="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 80)             │         3,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,981 (23.36 KB)

 Trainable params: 5,981 (23.36 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Compile the model

nn.compile(optimizer = "adam",loss ="binary_crossentropy",metrics =["accuracy"])


In [16]:
# Train the model

nn.fit(X_train_scaled,y_train, epochs =100,batch_size=10)

Epoch 1/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.7142 - loss: 0.5793
Epoch 2/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7262 - loss: 0.5570
Epoch 3/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7284 - loss: 0.5528
Epoch 4/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7343 - loss: 0.5479
Epoch 5/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7305 - loss: 0.5495
Epoch 6/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7260 - loss: 0.5508
Epoch 7/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7348 - loss: 0.5419
Epoch 8/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7327 - loss: 0.5485
Epoch 9/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7356 - loss: 0.5429
Epoch 10/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7347 - loss: 0.5433
Epoch 11/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7334 - loss: 0.5412
Epoch 12/100
2744/2

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.7265 - loss: 0.5730
Loss: 0.5730257630348206, Accuracy: 0.7265306115150452


In [18]:
# Export our model to HDF5 file

nn.save("my_model.h5")



# Compile, Train and Evaluate the Model with different nodes


In [19]:
#Define the  model -deep neural net, the number of input features and hidden nodes for each layers as follows
node1=128
node2=64
node3=64
node4=32
nn=tf.keras.models.Sequential()

# first hidden layer
nn.add(tf.keras.layers.Dense(units=128, input_dim=43, activation="relu"))
nn.add(tf.keras.layers.Dropout(0.2))

# second hidden layer
nn.add(tf.keras.layers.Dense(units=80, input_dim=43, activation="relu"))
nn.add(tf.keras.layers.Dropout(0.2))

#third hidden layer
nn.add(tf.keras.layers.Dense(units=64, input_dim=43, activation ="relu"))
nn.add(tf.keras.layers.Dropout(0.2))

#fourth hidden layer
nn.add(tf.keras.layers.Dense(units=32,input_dim=43,activation="relu"))
nn.add(tf.keras.layers.Dropout(0.2))

#output layer
nn.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

#check the structure of the model
nn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 128)            │         5,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 80)             │        10,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 80)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 64)             │         5,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,249 (90.82 KB)

 Trainable params: 23,249 (90.82 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
#compile and train
nn.compile(optimizer = "adam",loss ="binary_crossentropy",metrics =["accuracy"]) #compile
nn.fit(X_train_scaled,y_train, epochs =100,batch_size=10) # train

Epoch 1/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.6973 - loss: 0.6024
Epoch 2/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7270 - loss: 0.5673
Epoch 3/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7261 - loss: 0.5646
Epoch 4/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7289 - loss: 0.5590
Epoch 5/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7264 - loss: 0.5642
Epoch 6/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7307 - loss: 0.5554
Epoch 7/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7285 - loss: 0.5584
Epoch 8/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7287 - loss: 0.5548
Epoch 9/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7307 - loss: 0.5585
Epoch 10/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7317 - loss: 0.5528
Epoch 11/100
2744/2744 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7330 - loss: 0.5539
Epoch 12/100
2744/2

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.7277 - loss: 0.5627
Loss: 0.5627474784851074, Accuracy: 0.7276967763900757


In [22]:

# Initializing the neural network model
nn = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, input_dim=43, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(80, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Assuming binary classification
])

# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callback for early stopping
#early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

# Train the model
nn.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X, y))


Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7027 - loss: 0.5953 - val_accuracy: 0.4676 - val_loss: 140451.0312
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7260 - loss: 0.5564 - val_accuracy: 0.4676 - val_loss: 3616.8440
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7270 - loss: 0.5585 - val_accuracy: 0.4676 - val_loss: 28220.2461
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7321 - loss: 0.5507 - val_accuracy: 0.5324 - val_loss: 11257.2188
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7316 - loss: 0.5511 - val_accuracy: 0.4676 - val_loss: 5532.0234
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7348 - loss: 0.5464 - val_accuracy: 0.4676 - val_loss: 22387.2383
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7310 - loss: 0.5482 - val_accuracy: 0.5324 - val_loss: 18909.7383
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.735

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.7292 - loss: 0.5695
Loss: 0.5694606304168701, Accuracy: 0.7291545271873474


In [24]:

# Define the model architecture
nn = Sequential([
    Dense(128, input_dim=43, activation='relu'),
    Dropout(0.3),  # Increased dropout
    BatchNormalization(),
    Dense(128, activation='relu'),
    Dropout(0.3),  # Consistent dropout at each layer
    Dense(1, activation='sigmoid')
])

# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
nn.fit(
    X_train_scaled, y_train,
    epochs=200,  # Increased epochs
    batch_size=32,
    validation_data=(X, y))


Epoch 1/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6831 - loss: 0.6311 - val_accuracy: 0.4676 - val_loss: 21588.9668
Epoch 2/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7159 - loss: 0.5766 - val_accuracy: 0.4676 - val_loss: 78481.7734
Epoch 3/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7228 - loss: 0.5698 - val_accuracy: 0.4676 - val_loss: 80702.6094
Epoch 4/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7265 - loss: 0.5664 - val_accuracy: 0.4676 - val_loss: 46220.1406
Epoch 5/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7207 - loss: 0.5674 - val_accuracy: 0.4676 - val_loss: 86892.7578
Epoch 6/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7266 - loss: 0.5624 - val_accuracy: 0.4676 - val_loss: 4212.7778
Epoch 7/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7294 - loss: 0.5608 - val_accuracy: 0.4676 - val_loss: 55881.3203
Epoch 8/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.728

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.7286 - loss: 0.5633
Loss: 0.5632798075675964, Accuracy: 0.7285714149475098


In [26]:


nn = Sequential([
    Dense(256, input_dim=43, kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.3),
    
    Dense(256, kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.3),

    Dense(128, kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.3),

    Dense(1, activation='sigmoid')
])

nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
nn.fit(X_train_scaled, y_train, epochs=100, validation_data=(X, y))



Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.6869 - loss: 2.8976 - val_accuracy: 0.5324 - val_loss: 12699.4092
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7160 - loss: 0.6827 - val_accuracy: 0.4676 - val_loss: 3190.8591
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7191 - loss: 0.6367 - val_accuracy: 0.4676 - val_loss: 18912.6797
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7184 - loss: 0.6340 - val_accuracy: 0.5324 - val_loss: 7163.4639
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7198 - loss: 0.6378 - val_accuracy: 0.4676 - val_loss: 17199.6992
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.7220 - loss: 0.6387 - val_accuracy: 0.4676 - val_loss: 1198.1466
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.7242 - loss: 0.6350 - val_accuracy: 0.4676 - val_loss: 9806.5371
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.7124 -

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - 4ms/step - accuracy: 0.7248 - loss: 0.5855
Loss: 0.5855017900466919, Accuracy: 0.724781334400177
